# Lab 3

In this lab, we will practice Python's higher order functions, in particular, map(), filter() and reduce().


## Task 1

You are provided a list of service status updates scraped from an MTA information website. Each update may indicate <i>Good Service</i>, <i>Planned Work</i>, or <i>Delays</i> for one or more subway lines. Our first objective is to list all the lines that are running with <i>Delays</i>. To guide you through the process, we split the problem into smaller tasks.

In [12]:
from functools import reduce
import csv

# MapReduce
from mrjob.job import MRJob

In [2]:
# This is your input data, a list of subway line status.
# It is a list of string in a specific format

status = [
    '1,2,3 : Good Service',
    '4,5,6 : Delays',
    '7 : Good Service',
    'A,C : Good Service',
    'E : Planned Work',
    'G : Delays',
    'B,D,F,M : Good Service',
    'J,Z : Delays',
    'L : Good Service',
    'N,Q,R : Planned Work',
    'S : Good Service',
]

### Sub-Task 1

Please complete the lambda expression to filter only the status updates for the lines that run with <i>Delays</i>.

In [3]:
delayUpdates = list(filter(lambda x: 'Delays' in x , status))

delayUpdates
# After this, your delayUpdates should be
# ['4,5,6 : Delays', 'G : Delays', 'J,Z : Delays']

['4,5,6 : Delays', 'G : Delays', 'J,Z : Delays']

### Sub-Task 2

Please complete the lambda expression below to convert each status line into a list of subway lines, i.e. <b><i>'4,5,6 : Delays'</i></b> would become <b><i>['4','5','6']</i></b>

In [4]:
delayLineList = list(map(lambda x: x.split(' :')[0].split(','), delayUpdates))
delayLineList
# After this, your delayLineList should be
# [['4', '5', '6'], ['G'], ['J', 'Z']]

[['4', '5', '6'], ['G'], ['J', 'Z']]

### Sub-Task 3

Please complete the reduce command below to convert each the list of subway lists given in <i>delayLineList</i> into a single list of subway lines running with delay.

In [5]:
delayLines = reduce(lambda x, y: x+y, delayLineList) #reduce returns values, map and filter are generators
delayLines
# After this, your delayLines should be
# ['4', '5', '6', 'G', 'J', 'Z']

['4', '5', '6', 'G', 'J', 'Z']

### Sub-Task 4

Please complete the reduce command below to count the number of lines in <b>delayLines</b>.

In [6]:
delayLineCount = reduce(lambda x,y: x+1, delayLines, 0)
delayLineCount
# After this, your delayLineCount should be
# 6

6

## Task 2

In this excercise, we would like to expand the combined service updatse into separate updates for each subway line. For example, instead of having a single line <b>'1,2,3 : Good Service'</b> to indicate that line 1, 2, and 3 are in good service, we would like to convert that into 3 separate updates: <b>'1 : Good Service'</b>, <b>'2 : Good Service'</b>, and <b>'3 : Good Service'</b>.

You are tasked to write a chain of map(), filter(), and/or reduce() to convert the <b>status</b> variable into a list like below:

Please note that you may only use higher order functions without access to global variables. Your expression should contain only map(), filter() and/or reduce() and your custom function definitions.

In [7]:
status[:2]

['1,2,3 : Good Service', '4,5,6 : Delays']

In [8]:
# <ANY FUNCTION TO BE USED IN YOUR HOF>
def transform(x):
    y = x.split(' : ')
    lines =y[0].split(',')
    status = y[1]
    return dict((line , status) for line in lines)
    
'''  
    result =[]
    for line in lines:
        result.append(line + ' : ' + status)
    return result
'''
updates = reduce(lambda x, y: {**x,**y},map(transform,status),{})

#updates = transform(status[0])

#updates = reduce(lambda x, y: x+y, map(transform,status))
print(transform(status[0]))
# The expected value of updates is the list shown above
updates

{'1': 'Good Service', '2': 'Good Service', '3': 'Good Service'}


{'1': 'Good Service',
 '2': 'Good Service',
 '3': 'Good Service',
 '4': 'Delays',
 '5': 'Delays',
 '6': 'Delays',
 '7': 'Good Service',
 'A': 'Good Service',
 'C': 'Good Service',
 'E': 'Planned Work',
 'G': 'Delays',
 'B': 'Good Service',
 'D': 'Good Service',
 'F': 'Good Service',
 'M': 'Good Service',
 'J': 'Delays',
 'Z': 'Delays',
 'L': 'Good Service',
 'N': 'Planned Work',
 'Q': 'Planned Work',
 'R': 'Planned Work',
 'S': 'Good Service'}

In [9]:
x = {}
y = {1:2}
print(x.update(y))
x


None


{1: 2}

## Task 3

We would like to write an HOF expression to count the total number of trip activities involved each station. For example, if a rider starts a trip at station A and ends at station B, each station A and B will receive +1 count for  the trip. The output must be tuples, each consisting of a station name and a total count. A portion of the expected output are included below.

In [20]:
file = '../Files/citibike.csv'

with open(file, 'r') as fi:
    reader = csv.DictReader(fi)
    output3 = reduce(lambda x, y: x+y, map(lambda x: [x['start_station_name'], x['end_station_name']], reader), [])
    output3 = [(station, output1.count(station)) for station in set(output1)]

output3[:10]

[(('E 12 St & 3 Ave', 489), 1),
 (('Broadway & W 32 St', 431), 1),
 (('MacDougal St & Washington Sq', 179), 1),
 (('Allen St & Hester St', 309), 1),
 (('Old Slip & Front St', 121), 1),
 (('W 46 St & 11 Ave', 113), 1),
 (('Clark St & Henry St', 133), 1),
 (('E 4 St & 2 Ave', 390), 1),
 (('Fulton St & William St', 232), 1),
 (('Washington Pl & 6 Ave', 232), 1)]

In [30]:
from mrjob.job import MRJob, MRStep
import csv, sys
import mapreduce as mr

class MRTask3(MRJob):
    
    def mapper(self, _,  v1):
        yield v1['start_station_name'], 1
        yield v1['end_station_name'], 1
    
    def reducer(self, k2, v2s):
        yield k2, sum(v2s)

with open(file, 'r') as fi:
    reader = csv.DictReader(fi)
    output3 = list(mr.runJob(enumerate(reader), MRTask3(args=[])))
    
output3[:10]


[('1 Ave & E 15 St', 795),
 ('1 Ave & E 44 St', 219),
 ('10 Ave & W 28 St', 422),
 ('11 Ave & W 27 St', 354),
 ('11 Ave & W 41 St', 461),
 ('11 Ave & W 59 St', 242),
 ('12 Ave & W 40 St', 217),
 ('2 Ave & E 31 St', 588),
 ('2 Ave & E 58 St', 125),
 ('3 Ave & Schermerhorn St', 34)]

## Task 4

We would like to count the number of trips taken between pairs of stations. Trips taken from station A to station B or  from station B to station A are both counted towards the station pair A and B. *Please note that the station pair should be identified by station names, as a tuple, and **in lexical order**, i.e. **(A,B)** instead of ~~(B,A)~~ in this case*. The output must be tuples, each consisting of the station pair identification and a count. A portion of the expected output are included below. Please provide your HOF expression.

In [32]:
import csv
class MRTask4(MRJob):
    
    def mapper(self, k1,  v1):
        if v1['start_station_name'] < v1['end_station_name']:
            yield ((v1['start_station_name'] ,v1['end_station_name']),1)
        else:
            yield ((v1['end_station_name'],v1['start_station_name']),1)
    
    def reducer(self, k2, v2s):
        yield k2, sum(v2s)
        
with open(file, 'r') as fi:
    reader = csv.DictReader(fi)
    output3 = reduce(lambda x, y: x+y, map(lambda x: [(x['start_station_name'], x['end_station_name'])], reader), [])
    output3 = [(station, output3.count(station)) for station in set(output3)]

output3[:10]

[(('Forsyth St & Broome St', 'John St & William St'), 1),
 (('South St & Whitehall St', 'Allen St & Hester St'), 1),
 (('Warren St & Church St', 'Broad St & Bridge St'), 1),
 (('Canal St & Rutgers St', 'E 15 St & 3 Ave'), 1),
 (('Greenwich Ave & Charles St', 'E 16 St & 5 Ave'), 6),
 (('Broadway & W 49 St', 'W 42 St & 8 Ave'), 5),
 (('9 Ave & W 16 St', 'E 2 St & 2 Ave'), 1),
 (('E 24 St & Park Ave S', 'W 37 St & 10 Ave'), 1),
 (('Lafayette Ave & Fort Greene Pl', 'Forsyth St & Canal St'), 1),
 (('Lispenard St & Broadway', 'E 9 St & Avenue C'), 1)]

## Task 5

In this task, you are asked to compute the station with the most riders started from, per each gender of the *'Subscriber'* user. Meaning, what was the station name with the highest number of bike pickups for female riders, for male riders and for unknown riders.

The output will be a list of tuples, each includes a gender label (as indicated below) and another tuple consisting of a station name, and the total number of trips started at that station for that gender. The expected output are included below. Please provide your HOF expression below.

The label mapping for the gender column in citibike.csv is: (Zero=**Unknown**; 1=**Male**; 2=**Female**)

In [40]:
import csv
class MRTask5(MRJob):
    
    def mapper1(self, k1,  v1):
        if v1['start_station_name'] < v1['end_station_name']:
            yield ((v1['start_station_name'] ,v1['end_station_name']),1)
        else:
            yield ((v1['end_station_name'],v1['start_station_name']),1)
        #
    
    def reducer1(self, k2, v2s):
        yield k2, sum(v2s)

    def mapper2(self,k1,v1):
        gender, station = k1
        yield (gender, (station,v1))
        
    def reducer2(self, k2, v2s):
        gender = ('Unkonwns', 'Male','Female')
        yield (k2, len(v2s),sum(v2s))
        #yield gender[int(k2)], max(v2s, key=lambda x:x[1])#can use for loop to run multiple times
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(mapper=self.mapper2, reducer=self.reducer2)
        ]
with open(file, 'r') as fi:
    reader = csv.DictReader(fi)
    output5 = list(mr.runJob(enumerate(reader),MRTask5(args=[])))

output5

ValueError: invalid literal for int() with base 10: '1 Ave & E 15 St'

In [38]:
MRStep?

In [19]:
!python mr_word_count.py book.txt 2>/dev/null

"words"	38538
"lines"	5877
"chars"	246432


In [7]:
import csv
pd = {}

with open('sale_small.csv', 'r') as fi:
    reader = csv.reader(fi)
    next(reader)
    for record in reader:
        cid,pid,cost = (record[0],record[3],float(record[4]))
        if cid in pd:
            pd[pid][0] += cid
            pd[pid][1] += cost
        else:
            pd[pid] = [[cid],[cost]]
            
for pid,value in pd.items():
    print(pid, len(set(value[0])),sum(value[1]))

A 1 11.99
B 1 5.99
C 1 0.99
D 1 13.99
